# No-index performance analysis
Just to compare the results of our index we need to perform the same experiments in a brute force fashion. The brute force approach is also useful to discard some models, because with the index we can only get worse in term of Mean Average Precision.

# Import libraries

In [ ]:
!pip install scikit-learn-extra
from google.colab import drive
from pathlib import Path
import pandas as pd
import numpy as np
import concurrent.futures
import sklearn_extra
from PIL import Image
import sklearn 
import sklearn.metrics
from tqdm.auto import tqdm
import time
import csv
import os
import heapq

     |████████████████████████████████| 1.7 MB 5.1 MB/s 


# Connect to Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Set parameters

In [ ]:
n = 250
PERFORMANCE_FOLDER_FEATURE_EXTRACTION = '/content/drive/MyDrive/CV_Birds/performance/feature_extraction/no_index/ResNet152v2' 
PERFORMANCE_FOLDER_FINE_TUNING = '/content/drive/MyDrive/CV_Birds/performance/fine_tuning/no_index/ResNet152v2'
PERFORMANCE_FOLDER_MLP_ON_TOP = '/content/drive/MyDrive/CV_Birds/performance/MLP_on_top/no_index/ResNet152v2'

# Utility functions

In [ ]:
def create_annot(path):
  image_list = list(Path(path).glob('*/*.jpg'))
  # the identity name is in the path (the name of the parent directory)
  names_list = [i.parent.name for i in image_list]  # get the identity of each image
  # keep info in a pandas DataFrame
  annot = pd.DataFrame({'identity': names_list, 'image_path': image_list})
  return annot

def concatenate_annots(list_of_paths):
  concat_annot = pd.DataFrame()
  with concurrent.futures.ThreadPoolExecutor() as executor:
    annots = [executor.submit(create_annot, path) for path in list_of_paths]
    for annot in annots:
      new_annot = annot.result()
      concat_annot = concat_annot.append(new_annot, ignore_index = True)
    return concat_annot

def load_descriptors(path):
  with open(path, 'rb') as file:
    return np.load(file)

def concatenate_descriptors(list_of_paths):
  concat_descriptors = None
  with concurrent.futures.ThreadPoolExecutor() as executor:
    descriptors = [executor.submit(load_descriptors, path) for path in list_of_paths]
    for descriptor in descriptors:
      new_descriptor = descriptor.result()
      if concat_descriptors is None:
        concat_descriptors = new_descriptor
      else:
        concat_descriptors = np.concatenate([concat_descriptors, new_descriptor])
    return concat_descriptors

def cosine_distance(o1, o2):
  similarity = sklearn.metrics.pairwise.cosine_similarity(X=o1, Y=o2)
  return np.subtract(np.ones(similarity.shape), similarity)

def euclidean_distance(o1, o2):
  return sklearn.metrics.pairwise_distances(X=o1, Y=o2, metric='euclidean')

def k_nn_query(query_index, k, distance_metric = 'euclidean'):
  heap = []
  best_indexes = rank_distances = []
  query_descriptor = query_descriptors[query_index]
  for index in range(len(db_annot)):
    descriptor = db_descriptors[index]
    if distance_metric == 'euclidean':
      distance = euclidean_distance(query_descriptor.reshape(1,-1), descriptor.reshape(1,-1)).squeeze()
    elif distance_metric == 'cosine':
      distance = cosine_distance(query_descriptor.reshape(1,-1), descriptor.reshape(1,-1)).squeeze()
    else:
      return None, None  

    heapq.heappush(heap, (distance, index))

  smallest = heapq.nsmallest(k, heap)
  res = list(zip(*smallest))
  best_indexes = res[1]
  rank_distances = res[0]
  return best_indexes, rank_distances

def compute_ap_at_n(query_index, n, distance_metric = 'euclidean'):
  start_time = time.time()
  retrieved_indexes, _ = k_nn_query(query_index, n, distance_metric)
  end_time = time.time()
  query_identity = query_annot['identity'][query_index]
  GTP = len(db_annot.loc[db_annot['identity'] == query_identity])
  relevant = 0
  precision_summation = 0
  for k, id in enumerate(retrieved_indexes):
    if db_annot['identity'][id] == query_identity: # relevant result
      relevant = relevant + 1
      precision_at_k = relevant/(k+1)
      precision_summation = precision_summation + precision_at_k
  return precision_summation/GTP, end_time - start_time

def compute_aps (queries_indexes, distance_metric = 'euclidean'):
  aps_at_n = []
  query_times = []
  classes = []
  for i, query_index in enumerate(queries_indexes):
    print("Query number ", i, ", CLASS: ", query_annot['identity'][query_index])
    ap, t = compute_ap_at_n(query_index, n, distance_metric = distance_metric)
    classes.append(query_annot['identity'][query_index])
    aps_at_n.append(ap)
    query_times.append(t)
  return classes, aps_at_n, query_times

def save_results(dir, file_name, results):
  with open(os.path.join(dir, file_name + ".csv"), 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(["CLASS", "AP", "QUERY TIME"])
    # write the data
    for r in results:
      writer.writerow(r) 

# Test performance
We will now test the performance of the ResNet152v2 with a brute force approach. 

In [ ]:
query_annot = create_annot('/content/drive/MyDrive/CV_Birds/test')
query_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
...,...,...
1620,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1621,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1622,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1623,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...


To run our tests we select only the first image of each species within the test set. Please note that within the test set we have 5 images per species.

In [ ]:
queries_indexes = [x for x in range(325*5) if x%5 == 0]
print(query_annot['identity'][queries_indexes])

0             AFRICAN CROWNED CRANE
5                 AFRICAN FIREFINCH
10                        ALBATROSS
15             ALEXANDRINE PARAKEET
20                  AMERICAN AVOCET
                   ...             
1600       WILSONS BIRD OF PARADISE
1605                      WOOD DUCK
1610    YELLOW BELLIED FLOWERPECKER
1615                 YELLOW CACIQUE
1620        YELLOW HEADED BLACKBIRD
Name: identity, Length: 325, dtype: object


## Feature Extraction descriptors


In [ ]:
db_annot = create_annot('/content/drive/MyDrive/CV_Birds/train')
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
47327,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47328,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47329,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47330,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...


### ResNet50

#### Without distractor

In [ ]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/resnet_fe_features.npy')
db_descriptors.shape

(47332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/resnet50_fe_features.npy')
query_descriptors.shape

(1625, 2048)

##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_cosine_wo_distractor', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.0069448435589725245
Average Query Time: 19.28963907241821


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_cosine_wo_distractor.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.007557,19.912048
1,AFRICAN FIREFINCH,0.000314,19.195790
2,ALBATROSS,0.000736,20.305031
3,ALEXANDRINE PARAKEET,0.000379,19.537789
4,AMERICAN AVOCET,0.000441,19.080195
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000441,19.411022
321,WOOD DUCK,0.008479,19.366607
322,YELLOW BELLIED FLOWERPECKER,0.000108,19.625444
323,YELLOW CACIQUE,0.008393,19.089509


#### With distractor

In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/resnet_fe_features.npy', '/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/resnet50_fe_features.npy'])
db_descriptors.shape

(72332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/resnet50_fe_features.npy)
query_descriptors.shape

(1625, 2048)

##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_cosine_w_distractor', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.005960127039687729
Average Query Time: 21.520206193190354


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_cosine_w_distractor.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.007543,21.869399
1,AFRICAN FIREFINCH,0.000291,21.535254
2,ALBATROSS,0.000501,21.547009
3,ALEXANDRINE PARAKEET,0.000362,21.528182
4,AMERICAN AVOCET,0.000421,21.462070
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000426,21.641542
321,WOOD DUCK,0.008175,21.542091
322,YELLOW BELLIED FLOWERPECKER,0.000102,21.504295
323,YELLOW CACIQUE,0.008328,21.528363


### ResNet101

#### Without distractor

In [ ]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet101v2/resnet101_fe_features.npy')
db_descriptors.shape

(47332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet101v2/resnet101_fe_features.npy')
query_descriptors.shape

(1625, 2048)

###### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_cosine_wo_distractor', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.004816085731268082
Average Query Time: 13.86706734583928


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_cosine_wo_distractor.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.000063,13.934590
1,AFRICAN FIREFINCH,0.000055,14.114708
2,ALBATROSS,0.002615,13.634185
3,ALEXANDRINE PARAKEET,0.000000,13.700945
4,AMERICAN AVOCET,0.004926,13.592532
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000955,14.242536
321,WOOD DUCK,0.000427,14.419300
322,YELLOW BELLIED FLOWERPECKER,0.000061,13.936963
323,YELLOW CACIQUE,0.003930,14.042887


###ResNet152v2

#### Without distractor

In [ ]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/feature_extraction.npy')
db_descriptors.shape

(47332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/feature_extraction.npy')
query_descriptors.shape

(1625, 2048)

###### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_no_distractor_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.005651212303800769
Average Query Time: 13.5006614619035


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_no_distractor_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.000441,13.515887
1,AFRICAN FIREFINCH,0.007605,13.629154
2,ALBATROSS,0.008686,13.449941
3,ALEXANDRINE PARAKEET,0.000107,13.824955
4,AMERICAN AVOCET,0.000712,13.515467
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000746,13.509169
321,WOOD DUCK,0.002167,13.594899
322,YELLOW BELLIED FLOWERPECKER,0.000153,13.490915
323,YELLOW CACIQUE,0.001059,13.482093


##### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_no_distractor_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.0029004795134401204
Average Query Time: 10.20340964464041


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_no_distractor_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.000000,10.076458
1,AFRICAN FIREFINCH,0.000050,10.176174
2,ALBATROSS,0.000842,10.270569
3,ALEXANDRINE PARAKEET,0.000000,10.078368
4,AMERICAN AVOCET,0.000163,10.000636
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000462,10.211991
321,WOOD DUCK,0.002844,10.515118
322,YELLOW BELLIED FLOWERPECKER,0.000000,10.125777
323,YELLOW CACIQUE,0.000825,10.251473


#### With distractor

In [ ]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/feature_extraction.npy', '/content/drive/MyDrive/CV_Birds/features/distractor/ResNet152v2/feature_extraction.npy'])
db_descriptors.shape

(72332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/feature_extraction.npy')
query_descriptors.shape

(1625, 2048)

##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.004968566014353072
Average Query Time: 22.970515557802642


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.000223,21.596612
1,AFRICAN FIREFINCH,0.007563,21.560215
2,ALBATROSS,0.005442,21.380047
3,ALEXANDRINE PARAKEET,0.000105,24.426402
4,AMERICAN AVOCET,0.000626,21.689699
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000661,24.649814
321,WOOD DUCK,0.001911,24.308189
322,YELLOW BELLIED FLOWERPECKER,0.000141,24.801806
323,YELLOW CACIQUE,0.001054,23.639418


##### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_FEATURE_EXTRACTION, 'feature_extraction_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.002544972107464365
Average Query Time: 15.37566443003141


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FEATURE_EXTRACTION + '/feature_extraction_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.000000,15.355588
1,AFRICAN FIREFINCH,0.000044,15.370572
2,ALBATROSS,0.000290,15.399073
3,ALEXANDRINE PARAKEET,0.000000,15.312820
4,AMERICAN AVOCET,0.000150,15.450514
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.000447,15.327778
321,WOOD DUCK,0.002317,15.216356
322,YELLOW BELLIED FLOWERPECKER,0.000000,15.055472
323,YELLOW CACIQUE,0.000809,15.056837


## Fine Tuning descriptors
For what concerns the test done using fine-tuned models, they are all done considering the ResNet152v2. Thus, in the following section the base network is always the ResNet152v2

### Without distractor

In [ ]:
db_annot = create_annot('/content/drive/MyDrive/CV_Birds/train')
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
47327,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47328,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47329,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47330,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...


#### Global Average Pooling

In [ ]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/global_average_pooling_fine_tuning_4th.npy')
db_descriptors.shape

(47332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/global_average_pooling_fine_tuning_4th.npy')
query_descriptors.shape

(1625, 2048)

##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'global_average_pooling_fine_tuning_4th_no_distractor_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.4983536298572231
Average Query Time: 14.803395788486187


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/global_average_pooling_fine_tuning_4th_no_distractor_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.664254,14.373335
1,AFRICAN FIREFINCH,0.531997,14.063579
2,ALBATROSS,0.433291,13.965249
3,ALEXANDRINE PARAKEET,0.527854,13.963926
4,AMERICAN AVOCET,0.951620,14.120156
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.466832,14.422531
321,WOOD DUCK,0.539678,15.980927
322,YELLOW BELLIED FLOWERPECKER,0.092207,15.993528
323,YELLOW CACIQUE,0.554754,14.886257


##### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'global_average_pooling_fine_tuning_4th_no_distractor_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.3707759079284326
Average Query Time: 10.431788808382475


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/global_average_pooling_fine_tuning_4th_no_distractor_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.490365,11.043537
1,AFRICAN FIREFINCH,0.463194,10.218376
2,ALBATROSS,0.380963,10.524963
3,ALEXANDRINE PARAKEET,0.393806,10.517192
4,AMERICAN AVOCET,0.725209,10.363938
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.266250,10.380182
321,WOOD DUCK,0.526379,10.450012
322,YELLOW BELLIED FLOWERPECKER,0.044951,10.469019
323,YELLOW CACIQUE,0.457471,10.610842


#### One dense layer of 512 neurons and dropout

In [ ]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')
db_descriptors.shape

(47332, 512)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')
query_descriptors.shape

(1625, 512)

##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'OneDense512_Dropout_no_distractor_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.7683477116425814
Average Query Time: 14.474223973200871


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/OneDense512_Dropout_no_distractor_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.974857,14.662820
1,AFRICAN FIREFINCH,0.799997,13.976151
2,ALBATROSS,0.891434,13.885204
3,ALEXANDRINE PARAKEET,0.887709,14.664639
4,AMERICAN AVOCET,0.987317,14.414729
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.904037,14.391094
321,WOOD DUCK,0.837807,14.602926
322,YELLOW BELLIED FLOWERPECKER,0.395540,14.673997
323,YELLOW CACIQUE,0.936777,14.180743


##### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'OneDense512_Dropout_no_distractor_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.6953971006686744
Average Query Time: 10.104960355758667


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/OneDense512_Dropout_no_distractor_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.799216,10.427213
1,AFRICAN FIREFINCH,0.791040,10.409436
2,ALBATROSS,0.752350,11.096700
3,ALEXANDRINE PARAKEET,0.800542,10.031060
4,AMERICAN AVOCET,0.982473,10.196357
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.784198,10.120822
321,WOOD DUCK,0.835048,10.077660
322,YELLOW BELLIED FLOWERPECKER,0.398895,10.052656
323,YELLOW CACIQUE,0.901756,10.365149


### WIth distractor

In [ ]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


#### Global Average Pooling

In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/global_average_pooling_fine_tuning_4th.npy', '/content/drive/MyDrive/CV_Birds/features/distractor/ResNet152v2/global_average_pooling_fine_tuning_4th.npy'])
db_descriptors.shape

(72332, 2048)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/global_average_pooling_fine_tuning_4th.npy')
query_descriptors.shape

(1625, 2048)

##### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'global_average_pooling_fine_tuning_4th_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.3699753236738468
Average Query Time: 13.974797541544987


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/global_average_pooling_fine_tuning_4th_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.489594,14.394992
1,AFRICAN FIREFINCH,0.460734,14.558027
2,ALBATROSS,0.380391,14.338338
3,ALEXANDRINE PARAKEET,0.393753,14.328185
4,AMERICAN AVOCET,0.725209,14.285889
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.266250,13.922379
321,WOOD DUCK,0.523502,13.603261
322,YELLOW BELLIED FLOWERPECKER,0.044358,14.090874
323,YELLOW CACIQUE,0.457471,13.888602


##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'global_average_pooling_fine_tuning_4th_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.4967321311058971
Average Query Time: 21.609189572701087


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/global_average_pooling_fine_tuning_4th_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.659671,22.005408
1,AFRICAN FIREFINCH,0.529413,21.528520
2,ALBATROSS,0.431728,21.408374
3,ALEXANDRINE PARAKEET,0.511743,21.667012
4,AMERICAN AVOCET,0.951620,21.763314
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.466681,21.313735
321,WOOD DUCK,0.536385,21.410190
322,YELLOW BELLIED FLOWERPECKER,0.091477,21.321690
323,YELLOW CACIQUE,0.554079,22.934296


#### One dense layer of 512 neurons and dropout

In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/OneDense512_Dropout_fine_tuning.npy', '/content/drive/MyDrive/CV_Birds/features/distractor/ResNet152v2/OneDense512_Dropout_fine_tuning.npy'])
db_descriptors.shape

(72332, 512)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')
query_descriptors.shape

(1625, 512)

##### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'OneDense512_Dropout_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.6865849717766094
Average Query Time: 17.313917005979096


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING + '/OneDense512_Dropout_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.725999,16.511560
1,AFRICAN FIREFINCH,0.790997,16.117872
2,ALBATROSS,0.740697,16.250310
3,ALEXANDRINE PARAKEET,0.799964,16.656371
4,AMERICAN AVOCET,0.982473,17.090560
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.725408,18.264975
321,WOOD DUCK,0.832420,18.214250
322,YELLOW BELLIED FLOWERPECKER,0.397416,20.022932
323,YELLOW CACIQUE,0.897971,22.054394


##### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_FINE_TUNING, 'OneDense512_Dropout_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.764461939895279
Average Query Time: 19.94525253369258


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_FINE_TUNING+ '/OneDense512_Dropout_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.974726,20.735198
1,AFRICAN FIREFINCH,0.799938,20.296262
2,ALBATROSS,0.888852,20.287949
3,ALEXANDRINE PARAKEET,0.882358,20.057015
4,AMERICAN AVOCET,0.987317,19.833871
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.899934,19.817919
321,WOOD DUCK,0.832207,20.012264
322,YELLOW BELLIED FLOWERPECKER,0.394351,19.847483
323,YELLOW CACIQUE,0.936344,19.986971


## MLP on top

#### Without distractor

In [ ]:
db_annot = create_annot('/content/drive/MyDrive/CV_Birds/train')
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
47327,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47328,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47329,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...
47330,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/train/YELLOW H...


##### One dense layer of 512 neurons and dropout

In [ ]:
db_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')
db_descriptors.shape

(47332, 512)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')
query_descriptors.shape

(1625, 512)

###### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_MLP_ON_TOP, 'OneDense512_Dropout_no_distractor_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.5453579110789455
Average Query Time: 10.101914696326622


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_MLP_ON_TOP + '/OneDense512_Dropout_no_distractor_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.365754,10.259544
1,AFRICAN FIREFINCH,0.609640,10.069178
2,ALBATROSS,0.506010,9.924977
3,ALEXANDRINE PARAKEET,0.620176,9.971789
4,AMERICAN AVOCET,0.933808,10.035733
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.726838,9.886036
321,WOOD DUCK,0.784809,10.145702
322,YELLOW BELLIED FLOWERPECKER,0.131385,10.175039
323,YELLOW CACIQUE,0.690218,9.928082


###### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_MLP_ON_TOP, 'OneDense512_Dropout_no_distractor_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.6043010052506272
Average Query Time: 13.141706600922804


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_MLP_ON_TOP + '/OneDense512_Dropout_no_distractor_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.926592,13.335992
1,AFRICAN FIREFINCH,0.621191,13.125925
2,ALBATROSS,0.643405,13.791077
3,ALEXANDRINE PARAKEET,0.625579,13.180200
4,AMERICAN AVOCET,0.978793,13.248659
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.850319,13.150941
321,WOOD DUCK,0.780188,13.295351
322,YELLOW BELLIED FLOWERPECKER,0.126440,13.083058
323,YELLOW CACIQUE,0.814693,13.137298


#### With distractor

In [ ]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


##### One dense layer of 512 neurons and dropout

In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/ResNet152v2/OneDense512_Dropout_feature_extraction.npy', '/content/drive/MyDrive/CV_Birds/features/distractor/ResNet152v2/OneDense512_Dropout_feature_extraction.npy'])
db_descriptors.shape

(72332, 512)

In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')
query_descriptors.shape

(1625, 512)

###### Euclidean distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='euclidean')
save_results(PERFORMANCE_FOLDER_MLP_ON_TOP, 'OneDense512_Dropout_euclidean', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.5416694492780932
Average Query Time: 15.171908584741447


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_MLP_ON_TOP + '/OneDense512_Dropout_euclidean.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.358664,16.063233
1,AFRICAN FIREFINCH,0.609432,15.143804
2,ALBATROSS,0.505847,15.741786
3,ALEXANDRINE PARAKEET,0.619437,15.103472
4,AMERICAN AVOCET,0.933791,15.351322
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.726822,15.137998
321,WOOD DUCK,0.779576,15.171225
322,YELLOW BELLIED FLOWERPECKER,0.131206,15.253132
323,YELLOW CACIQUE,0.690218,14.980313


###### Cosine distance

In [ ]:
classes, aps_at_n, query_times = compute_aps (queries_indexes, distance_metric='cosine')
save_results(PERFORMANCE_FOLDER_MLP_ON_TOP, 'OneDense512_Dropout_cosine', zip(classes, aps_at_n, query_times))

In [ ]:
mAP_at_n = np.mean(aps_at_n, axis=0)
average_query_time = np.mean(query_times, axis=0)
print(f'Mean Average Precision at {n}: {mAP_at_n}')
print(f'Average Query Time: {average_query_time}')

Mean Average Precision at 250: 0.601031660488315
Average Query Time: 20.039991641411415


In [ ]:
pd.read_csv(PERFORMANCE_FOLDER_MLP_ON_TOP + '/OneDense512_Dropout_cosine.csv') 

,CLASS,AP,QUERY TIME
0,AFRICAN CROWNED CRANE,0.926446,19.972075
1,AFRICAN FIREFINCH,0.620818,20.143416
2,ALBATROSS,0.633326,20.186301
3,ALEXANDRINE PARAKEET,0.622254,19.894601
4,AMERICAN AVOCET,0.978793,20.011874
...,...,...,...
320,WILSONS BIRD OF PARADISE,0.849470,19.971931
321,WOOD DUCK,0.777656,21.516886
322,YELLOW BELLIED FLOWERPECKER,0.126289,19.851404
323,YELLOW CACIQUE,0.814001,19.886149
